# 02. Moving to a Framework: PydanticAI

In tutorial 01 we wired up a bare-bones agent manually. However, all the engineering around the agent, tools and memory can be offloaded to an Agentic framework. Agentic frameworks like [Pydantic AI](https://ai.pydantic.dev/) or [LangChain](https://www.langchain.com/) simplify this by turning LLMs into structured, stateful agents that can:
- **Reason** → generate thoughts or plans.
- **Act** → call functions or tools safely.
- **Remember** → maintain state across steps.
- **Validate** → ensure model responses follow predefined schemas automatically.

Essentially, they make the “LLM + tools + memory” loop *modular* and *robust*.

[Pydantic AI](https://ai.pydantic.dev/) provides:
- **Automatic JSON validation** via Pydantic models.  
- **Cleaner syntax** — you define structure, not glue code.  
- **Built-in model orchestration** with minimal setup.  
- **Extensible memory and reasoning controls** for larger agent systems.


This transition will help you focus on **designing intelligent behaviors**, not wiring model calls.  We'll keep the examples compact so you can focus on the core ideas, while also mapping each topic back to the official documentation so you can dig deeper on your own. Let's extend our Agentic AI knowledge to include:

- How the core `Agent` API maps to models, prompts, and run results.
- How to register tools, inject dependencies, and read the `RunContext`.
- How to add structured outputs, guardrails (validation + redaction), and retries.
- How to stream responses and log the conversation with Logfire.
- How to glue everything together in a realistic concierge use-case.
- Where to find each concept inside the [PydanticAI docs](https://ai.pydantic.dev/).


## Documentation roadmap
PydanticAI's site is organized by concept. As you experiment in this notebook, keep these reference pages open:

| Topic | Docs quick link | Why it matters |
| --- | --- | --- |
| Agents & models | [`/agents/`](https://ai.pydantic.dev/agents/) | Explains the `Agent` constructor, run methods, and model adapters. |
| Tools & dependencies | [`/tools/`](https://ai.pydantic.dev/tools/) | Shows the `@agent.tool` decorator, dependency injection, and context helpers. |
| Results & streaming | [`/output/`](https://ai.pydantic.dev/output/) | Documents the output, streaming iterators, and metadata on each run. |
| Observability | [`/logfire/`](https://ai.pydantic.dev/logfire/) | Walks through Logfire setup, spans, and structured traces. |

We'll point back to these sections throughout the tutorial so you can connect code to the syntax explained in the docs.

I highly recommend creating a free [Logfire](https://logfire.pydantic.dev/) account for rich traces.


## 1. Meet `Agent`
PydanticAI wraps model calls inside an `Agent`. You configure the model, give it a concise system prompt, and call `run_sync` to execute a turn.

We'll use a concierge persona throughout the notebook.

### Anatomy of `Agent(...)`
The constructor mirrors the [agent configuration docs](https://ai.pydantic.dev/agents/#agent-constructor):

- `model`: A model adapter such as `OpenAIModel`, `AnthropicModel`, or `LiteLLMModel`. Each adapter exposes provider-specific settings via keyword arguments.
- `system_prompt`: A concise instruction that frames every conversation turn.
- `deps_type`: (Optional) A Pydantic model describing the structured dependencies your tools need.
- `result_type`: (Optional) A Pydantic model that constrains the final answer.
- `tools`: (Optional) A list of callables registered ahead of time. You can also decorate functions with `@agent.tool` as shown later.

Every `Agent` offers synchronous (`run_sync`) and asynchronous (`run`) helpers that return a [`RunResult`](https://ai.pydantic.dev/results/#runresult). We'll use the sync variants to keep the tutorial focused. We also use OpenRouter provider by explicitly specifying `openrourter:` in the model argument.

In [53]:
from pydantic_ai import Agent

from dotenv import load_dotenv
import nest_asyncio
nest_asyncio.apply()

load_dotenv()

concierge = Agent(
    model="openrouter:qwen/qwen3-next-80b-a3b-instruct",
    system_prompt="You are a friendly city concierge. Keep replies short, positive, and specific to Berlin.",
)

intro = concierge.run_sync("Welcome a visitor to Berlin in one sentence.")
print(intro.output)

19:57:01.284 concierge run
19:57:01.285   chat qwen/qwen3-next-80b-a3b-instruct
Welcome to Berlin—where history, culture, and vibrant energy come alive at every corner!


`Agent` accepts the model backend (OpenAI, Anthropic, etc.), a system prompt, and optional extras. `run_sync` sends the user message, waits for the model, and returns a `RunResult` with metadata like `response_text`, tool traces, and tokens.

`RunResult` carries the `response_text`, a structured `data` attribute when `result_type` is used, token usage, tool call history, and helper methods like `.all_messages()` to inspect the full transcript. Inspect it in the notebook or read the [results guide](https://ai.pydantic.dev/results/) for the complete schema. For instance, for the above call, we see the reasoning tokens as well for the response.

In [13]:
intro.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='You are a friendly city concierge. Keep replies short, positive, and specific to Berlin.', timestamp=datetime.datetime(2025, 11, 3, 13, 37, 21, 267783, tzinfo=datetime.timezone.utc)), UserPromptPart(content='Welcome a visitor to Berlin in one sentence.', timestamp=datetime.datetime(2025, 11, 3, 13, 37, 21, 267783, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[ThinkingPart(content='\nOkay, the user wants me to welcome a visitor to Berlin in one sentence. Let me think about the key elements that make Berlin unique. There\'s the historical landmarks like the Brandenburg Gate and Berlin Wall, the vibrant culture, maybe the arts scene. Also, the city\'s mix of old and new, modern architecture alongside historical sites.\n\nI should make it friendly and positive. Use words that evoke warmth and excitement. Maybe mention the energy of the city, the blend of past and present. Avoid being too generic. Need to keep it concise, one sentence

`Agent` also allows [dynamic system prompts](https://ai.pydantic.dev/agents/#system-prompts) where we can change the system prompt based on the input context. We do this by providing a `RunContext` object that exposes the dependencies plus request metadata.

In [54]:
from pydantic_ai import RunContext

concierge = Agent(
    model="openrouter:qwen/qwen3-next-80b-a3b-instruct",
)

@concierge.system_prompt
def add_city(ctx: RunContext[str]) -> str:
    return f"You are a friendly city concierge. Keep replies short, positive, and specific to {ctx.deps}."

intro = concierge.run_sync("Give an itinerary for a tourist for a single day in one sentence.", deps="Moscow")
intro.all_messages()

19:57:10.216 concierge run
19:57:10.231   chat qwen/qwen3-next-80b-a3b-instruct


[ModelRequest(parts=[SystemPromptPart(content='You are a friendly city concierge. Keep replies short, positive, and specific to Moscow.', timestamp=datetime.datetime(2025, 11, 3, 14, 27, 10, 231090, tzinfo=datetime.timezone.utc)), UserPromptPart(content='Give an itinerary for a tourist for a single day in one sentence.', timestamp=datetime.datetime(2025, 11, 3, 14, 27, 10, 231090, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[TextPart(content='Start at Red Square, explore the Kremlin and St. Basil’s, enjoy lunch at GUM, stroll through Arbat Street, and end with sunset views from Sparrow Hills.')], usage=RequestUsage(input_tokens=46, output_tokens=37), model_name='qwen/qwen3-next-80b-a3b-instruct', timestamp=datetime.datetime(2025, 11, 3, 14, 27, 11, tzinfo=TzInfo(0)), provider_name='openrouter', provider_details={'finish_reason': 'stop'}, provider_response_id='gen-1762180031-jfcS5f57CdDA4B75fqQr', finish_reason='stop')]

We see the system prompt changed as per the input. We can do something similar for other aspects such as adding today's date. We can also get all messages in JSON format.

In [55]:
from datetime import date
import json

@concierge.system_prompt
def add_the_date() -> str:  
    return f'The date is {date.today()}.'

intro = concierge.run_sync("Give an itinerary for a tourist for a single day in one sentence.", deps="New Delhi")
json.loads(intro.all_messages_json())

19:57:17.029 concierge run
19:57:17.029   chat qwen/qwen3-next-80b-a3b-instruct
19:57:18.156   chat qwen/qwen3-next-80b-a3b-instruct


[{'parts': [{'content': 'You are a friendly city concierge. Keep replies short, positive, and specific to New Delhi.',
    'timestamp': '2025-11-03T14:27:17.029166Z',
    'dynamic_ref': None,
    'part_kind': 'system-prompt'},
   {'content': 'The date is 2025-11-03.',
    'timestamp': '2025-11-03T14:27:17.029166Z',
    'dynamic_ref': None,
    'part_kind': 'system-prompt'},
   {'content': 'Give an itinerary for a tourist for a single day in one sentence.',
    'timestamp': '2025-11-03T14:27:17.029166Z',
    'part_kind': 'user-prompt'}],
  'instructions': None,
  'kind': 'request'},
 {'parts': [],
  'usage': {'input_tokens': 59,
   'cache_write_tokens': 0,
   'cache_read_tokens': 0,
   'output_tokens': 0,
   'input_audio_tokens': 0,
   'cache_audio_read_tokens': 0,
   'output_audio_tokens': 0,
   'details': {}},
  'model_name': 'qwen/qwen3-next-80b-a3b-instruct',
  'timestamp': '2025-11-03T14:27:18Z',
  'kind': 'response',
  'provider_name': 'openrouter',
  'provider_details': {'finish_

## 2. Tools + Dependencies: Local Concierge Use-Case
Agents become useful once they can call real data. We'll model a concierge that knows Berlin restaurants and events stored in plain Python dictionaries.

PydanticAI lets you declare a dependency schema (`deps_type`) and register tool functions with the `@agent.tool` decorator. Tools receive a `RunContext` that exposes the dependencies plus request metadata.

### Why dependencies matter
The docs describe dependencies as a way to [share stateful resources](https://ai.pydantic.dev/tools/#dependencies) (database clients, configuration, caches) without global variables. Defining `deps_type` with Pydantic gives you:

1. **Validation** – Data loaded into the dependency object is checked once, so tools always see a consistent shape.
2. **Auto-complete** – Type hints make editor tooling and docstrings clearer.
3. **Runtime access** – Inside a tool, `ctx.deps` exposes that object alongside metadata such as `ctx.user_message` and `ctx.conversation`.

We'll start with an in-memory dataset, but the same pattern works with network or database clients.


In [14]:
from typing import Dict, List

from pydantic import BaseModel

class ConciergeDeps(BaseModel):
    events: Dict[str, List[str]]
    restaurants: Dict[str, List[str]]

    def lookup(self, city: str) -> dict:
        key = city.lower()
        return {
            "events": self.events.get(key, []),
            "restaurants": self.restaurants.get(key, []),
        }

berlin_data = ConciergeDeps(
    events={
        "berlin": ["Saturday street food market in Kreuzberg", "Museum Island late-night opening", "Spree sunset boat cruise"],
    },
    restaurants={
        "berlin": ["Five Elephant Coffee Roastery", "Markthalle Neun vendors", "Mustafas Gemüse Kebap"],
    },
)

In [27]:
from pydantic_ai import RunContext

guide = Agent(
    model="openrouter:qwen/qwen3-next-80b-a3b-instruct",
    system_prompt=(
        "You are a Berlin concierge. Use the `fetch_local_options` tool before planning a day so you stay factual. "
        "Summaries should include morning, afternoon, and evening suggestions."
    ),
    deps_type=ConciergeDeps,
)

@guide.tool
def fetch_local_options(ctx: RunContext[ConciergeDeps], city: str) -> dict:
    """Return curated restaurants and events for the requested city."""
    return ctx.deps.lookup(city)

day_plan = guide.run_sync(
    "Plan a Saturday in Berlin for a family with teenagers. Include meals and activities.",
    deps=berlin_data,
)
print(day_plan.output)

Here’s a fun, balanced Saturday plan in Berlin tailored for a family with teenagers — mixing food, culture, and movement:

### **Morning**
**Breakfast at Five Elephant Coffee Roastery**  
Start the day with specialty coffee and freshly baked pastries at this beloved local roastery in Neukölln. Their avocado toast and vegan muffins are crowd-pleasers, and the cozy, industrial-chic vibe is Instagram-ready — perfect for teens.  

**Activity: Street Food Market in Kreuzberg**  
Head to the Saturday street food market at Markthalle Neun (just a short U-Bahn ride away). Teens will love the global flavors — try Korean tacos, gourmet burgers, or Portuguese custard tarts. Wander through the vibrant stalls, snap photos, and taste-test together.  

### **Afternoon**
**Museum Island — Late-Night Opening (start early!)**  
Browse the Pergamon Museum or the Altes Museum on Museum Island. The late-night opening means fewer crowds and a cool, atmospheric vibe. Teens often enjoy the ancient artifacts —

### Inspecting tool calls
Tool executions are recorded on the result. You can explore them interactively or via code:

Each `ToolCall` matches the structure described in the [tooling reference](https://ai.pydantic.dev/tools/#tool-calls), including start/end timestamps and any raised errors.

Let's break down all interactions below. We see:

- Agent calls tool with the argument city as "Berlin"
- uses the result to provide an informed response.

In [ ]:
json.loads(day_plan.all_messages_json())

[{'parts': [{'content': 'You are a Berlin concierge. Use the `fetch_local_options` tool before planning a day so you stay factual. Summaries should include morning, afternoon, and evening suggestions.',
    'timestamp': '2025-11-03T13:47:14.738937Z',
    'dynamic_ref': None,
    'part_kind': 'system-prompt'},
   {'content': 'Plan a Saturday in Berlin for a family with teenagers. Include meals and activities.',
    'timestamp': '2025-11-03T13:47:14.738937Z',
    'part_kind': 'user-prompt'}],
  'instructions': None,
  'kind': 'request'},
 {'parts': [{'tool_name': 'fetch_local_options',
    'args': '{"city": "Berlin"}',
    'tool_call_id': 'call_3c9d2254767b45468ec8d3',
    'id': None,
    'part_kind': 'tool-call'}],
  'usage': {'input_tokens': 208,
   'cache_write_tokens': 0,
   'cache_read_tokens': 0,
   'output_tokens': 20,
   'input_audio_tokens': 0,
   'cache_audio_read_tokens': 0,
   'output_audio_tokens': 0,
   'details': {}},
  'model_name': 'qwen/qwen3-next-80b-a3b-instruct',
  '

`RunContext` offers more than dependencies. Explore `ctx.user_message`, `ctx.deps`, and `ctx.all_messages` (the running transcript) inside tools to tailor behavior—e.g., short-circuiting repeat lookups. The [context section of the docs](https://ai.pydantic.dev/tools/#tool-context) lists every field you can rely on.


During the run the model can call `fetch_local_options`. The framework handles JSON arguments, dependency injection, and logging. Tool outputs are automatically threaded back into the model's context so later messages can reference them.

## 3. Model retry + Logfire tracing
Real traffic can be noisy: models may time out or return malformed JSON. [`ModelRetry`](https://ai.pydantic.dev/api/exceptions/#pydantic_ai.exceptions.ModelRetry) replays a call with the same inputs. Combining it with Logfire gives you deep observability—every attempt (and tool call) shows up in the Logfire UI.

The snippet below mirrors the [Logfire integration guide](https://ai.pydantic.dev/logfire/). 

In [56]:
import logfire
from pydantic_ai import ModelRetry

logfire.configure()
logfire.instrument_pydantic_ai()

guide = Agent(
    model="openrouter:qwen/qwen3-next-80b-a3b-instruct",
    system_prompt=(
        "You are a Berlin concierge. Use the `fetch_local_options` tool before planning a day so you stay factual. "
        "Summaries should include morning, afternoon, and evening suggestions."
    ),
    retries=4,
    deps_type=ConciergeDeps
)

@guide.tool
def fetch_local_options(ctx: RunContext[ConciergeDeps], city: str) -> dict:
    """Return curated restaurants and events for the requested city."""
    if city.lower() not in ['berlin']:
        ModelRetry('no data for this city')
    return ctx.deps.lookup(city)

winter_plan = guide.run_sync(
    "It's February and cold. Suggest an indoor-focused Berlin Saturday.",
    deps=berlin_data,
)

print(winter_plan.output)

Logfire project URL: ]8;id=66884;https://logfire-eu.pydantic.dev/shreshthtuli/agenticai\https://logfire-eu.pydantic.dev/shreshthtuli/agenticai]8;;\

20:03:09.799 guide run
20:03:09.801   chat qwen/qwen3-next-80b-a3b-instruct
20:03:11.186   running 1 tool
20:03:11.186     running tool: fetch_local_options
20:03:11.186   chat qwen/qwen3-next-80b-a3b-instruct
Here’s your cozy, indoor-focused Berlin Saturday, perfect for February’s chill:

**Morning: Warm Start**  
Begin at **Five Elephant Coffee Roastery** in Mitte—sip expertly brewed coffee and enjoy a buttery pastry in their minimalist, steamy interior. It’s the perfect place to shake off the morning cold.

**Afternoon: Culture & Warmth**  
Head to **Museum Island** for its special late-night opening (yes, even on Saturday!). Explore the Pergamon Museum or the Old National Gallery at your leisure, surrounded by world-class art and architecture—no freezing winds, just quiet wonder.

**Evening: Cozy Fare & Local Vibes**  
Dinner at **Markthalle Neun** in Kreuzberg. While the street food market isn’t active, the halls are still bustling with warm, artisanal eateries. Try hearty German 

This is what it looks like

![image](assets/logfire-ss.png)

## 4. Streaming replies
LLMs can stream partial text so you don't block users. `run_stream` returns an iterator; use `stream_text()` to yield deltas as soon as they arrive. The [streaming guide](https://ai.pydantic.dev/output/#streaming-text) shows additional helpers for partial structured results.


In [37]:
async with guide.run_stream(
        "Stream a warm two-sentence welcome for someone arriving in Berlin this evening.",
        deps=berlin_data,
    ) as stream:
        async for token in stream.stream_text():
            print(token, flush=True)

19:33:43.707 guide run
19:33:43.708   chat qwen/qwen3-next-80b-a3b-instruct
Welcome to Berlin, where history whispers through cobblestone streets and modern energy pulses in every corner. Whether
Welcome to Berlin, where history whispers through cobblestone streets and modern energy pulses in every corner. Whether you’re here for the art, the food, or the
Welcome to Berlin, where history whispers through cobblestone streets and modern energy pulses in every corner. Whether you’re here for the art, the food, or the midnight vibes, the city is ready to embrace you.


We can also stream text as deltas rather than entire text in each item.

In [39]:
async with guide.run_stream(
        "Stream a warm two-sentence welcome for someone arriving in Berlin this evening.",
        deps=berlin_data,
    ) as stream:
        async for token in stream.stream_text(delta=True):
            print(token, end="", flush=True)

19:34:41.405 guide run
19:34:41.407   chat qwen/qwen3-next-80b-a3b-instruct
Willkommen in Berlin! Die Stadt erwacht erst so richtig abends – von goldenen Lichtern gesprenkelt, lädt sie ein, durch enge Gassen zu schlendern und die Luft mit einem frischen Berliner Bier zu genießen. Genießen Sie Ihren ersten Abend, als ob Sie schon immer hiergehört hätten.

## 5. Guardrails: validation + redaction
Structured results reduce hallucinations. Pass a Pydantic model as `result_type` and the agent will coerce the response into that schema (retrying on validation errors). You can mark sensitive fields with `SecretStr` so that logs mask them automatically. 


In [46]:
from typing import Literal, Optional
from pydantic import BaseModel, EmailStr, Field, PositiveInt, SecretStr, field_validator, field_serializer
from pydantic.networks import validate_email
from pydantic_ai import Agent

class DinnerReservation(BaseModel):
    guest_name: str = Field(max_length=60)
    guest_email: SecretStr  # store securely
    party_size: PositiveInt = Field(le=8)
    preferred_time: Literal["17:00", "18:00", "19:00", "20:00"]
    special_requests: Optional[str] = Field(default=None, max_length=200)

    # validate the inner string as an email
    @field_validator("guest_email")
    @classmethod
    def validate_email(cls, v: SecretStr):
        # this will raise if invalid
        validate_email(v.get_secret_value())
        return v

    # ensure JSON/logs show masked value
    @field_serializer("guest_email", when_used="json")
    def dump_masked(self, v: SecretStr):
        return "**********"  # or return EmailStr(v.get_secret_value()).obfuscated

reservations = Agent(
    model="openrouter:qwen/qwen3-next-80b-a3b-instruct",
    system_prompt=(
        "Extract structured dinner reservations from chatty guest messages. "
        "If details are missing, make a polite best guess."
    ),
    output_type=DinnerReservation,
    retries=3
)

guest_message = """
Hi! I'm Alex visiting with 3 coworkers this Friday. Could you book something cozy around 7pm?
My email is alex@example.com and we'd love a vegetarian-friendly spot.
"""

reservation = reservations.run_sync(guest_message)

# Internal use (actual email)
real_email = reservation.output.guest_email.get_secret_value()

# Safe for logs/JSON
print("Safe JSON:", reservation.output.model_dump_json())
print("Real email:", real_email)


19:47:56.235 reservations run
19:47:56.236   chat qwen/qwen3-next-80b-a3b-instruct
Safe JSON: {"guest_name":"Alex","guest_email":"**********","party_size":4,"preferred_time":"19:00","special_requests":"Vegetarian-friendly options"}
Real email: alex@example.com


Redacted fields still exist in `reservation.output`, but when Logfire (or standard logging) serializes the object the value is replaced with `***`. If the model fails validation (e.g., returns an invalid email), PydanticAI automatically retries up to the configured limit before surfacing a `ValidationError`.

## 6. Putting it all together

We have shown how Pydantic AI makes it easy to define input structure via `deps_type`, output structure via `output_type` with dynamic system prompts `@agent.system_prompt` and tools `@agent.tool` decorators. We also show how to use context and dependencies effectively. 

To adapt this concierge into your own project:
1. Swap the dictionaries with a real API client or database dependency.
2. Add more tools—weather lookup, transit times, ticket booking—and observe how the agent decides between them.
3. Combine streaming with a UI (FastAPI, Streamlit) to deliver real-time concierge experiences.
4. Turn on hosted Logfire to collect traces from production traffic.
5. Explore advanced features like [parallel tool calls](https://ai.pydantic.dev/tools-advanced/#parallel-tool-calls-concurrency) and [memory primitives](https://ai.pydantic.dev/builtin-tools/#memory-tool) once you're comfortable with the basics.
